read dde from kn.her.all, first column includes two single mutations aa and site information, second column is dde, third is DE_double, fourth is first de, fifth is second de

# 1. read dde from file


In [1]:
import pandas as pd
# Try reading the file with space as a delimiter
df = pd.read_csv('kn.her2.all', delim_whitespace=True, header=None, names=['Mutation', 'DDE','DE_double', 'First_DE', 'Second_DE'])
# Split the 'Mutation' column into 'First_mutation' and 'Second_mutation'
df[['First_mutation', 'Second_mutation']] = df['Mutation'].str.split('-', expand=True)
# Display the DataFrame
print(df)


           Mutation       DDE  DE_double  First_DE  Second_DE First_mutation  \
0           A1B-B2A  0.012168 -20.532304 -8.380550 -12.211337            A1B   
1           A1B-B2C -0.000908 -16.498453 -8.380550  -8.179655            A1B   
2           A1B-B2D  0.000125 -16.658839 -8.380550  -8.339867            A1B   
3           A1B-A3B  0.008785 -14.738873 -8.380550  -6.419735            A1B   
4           A1B-A3C  0.008937 -13.811957 -8.380550  -5.493224            A1B   
...             ...       ...        ...       ...        ...            ...   
310072  B262C-A263C  0.390912 -16.406621 -8.433813  -8.236725          B262C   
310073  B262C-A263D  0.391073 -16.553047 -8.433813  -8.383161          B262C   
310074  B262D-A263B  0.267580 -15.236453 -8.310417  -7.223255          B262D   
310075  B262D-A263C  0.001986 -16.321536 -8.310417  -8.236725          B262D   
310076  B262D-A263D  0.002057 -16.467874 -8.310417  -8.383161          B262D   

       Second_mutation  
0             

# 2. DDE exploration

## 2.1 Find strongest dde pairs

In [ ]:
# Find the strongest pairs based on absolute value of DDE
strongest_pairs = df.loc[df['DDE'].abs().nlargest(5).index]

# Display the strongest pairs
# print(strongest_pairs)

# Extract numeric parts of the mutation sites
strongest_pairs['First_site'] = strongest_pairs['First_mutation'].str.extract('(\d+)').astype(int)
strongest_pairs['Second_site'] = strongest_pairs['Second_mutation'].str.extract('(\d+)').astype(int)

# Filter pairs where the mutation sites are not adjacent
non_adjacent_strongest_pairs = strongest_pairs[abs(strongest_pairs['First_site'] - strongest_pairs['Second_site']) > 1]

# Display the filtered pairs
print(non_adjacent_strongest_pairs)

           Mutation      DDE  DE_double  First_DE  Second_DE First_mutation  \
242203  C140D-D148B  8.50879 -11.219437 -6.705909  -4.577816          C140D   
244997  D143A-D230C  6.37754 -19.363818 -8.272259  -9.935544          D143A   
259045  B157A-D160B  6.08820 -11.517552 -4.094523  -7.406373          B157A   

       Second_mutation  First_site  Second_site  
242203           D148B         140          148  
244997           D230C         143          230  
259045           D160B         157          160  


## 2.2 J matrix and delta e definition

### 2.2.1 J matrix

In [3]:
import numpy as np

#dictionary of J matrix
J_dict = {}

# Load the J matrix from the downloaded file
J = np.load('J.npy')

row = 0
for pos1 in range(1, 264):
    for pos2 in range(pos1 + 1, 264):

        for i, aa1 in enumerate(['A', 'B', 'C', 'D']):
            for j, aa2 in enumerate(['A', 'B', 'C', 'D']):
                col = i*4+j


                J_dict[(pos1, pos2, aa1, aa2)] = J[row, col]
                J_dict[(pos2, pos1, aa2, aa1)] = J[row, col]
            # print(f"Row {row}: J[{row}, {col}] = {J[row, col]} for positions ({pos1}, {pos2}) with amino acids ({aa1}, {aa2})")
        row += 1

print(f"Dictionary created with {len(J_dict)} entries")

Dictionary created with 1102496 entries


### 2.2.2 load sequeces

In [4]:
with open('in.reduce4.seq', 'r') as f:
    sequence = f.read().strip()
sequence_list = sequence.split('\n')
print(f"Loaded sequence with {len(sequence_list)} lines")

Loaded sequence with 1220 lines


### 2.2.3 define delta e

In [5]:
# Define delta E calculation
def calculate_delta_e(position, old_amino_acid, new_amino_acid, sequence_list, J_dict):
    # E(old_amino_acid)
    energy_old = 0
    for other_pos in range(1, 264):
        if other_pos == position:
            continue
        other_aa = sequence_list[0][other_pos - 1]  # Access the first sequence in sequence_list
        energy_old += J_dict.get((position, other_pos, old_amino_acid, other_aa), 0)

    # E(new_amino_acid)
    energy_new = 0
    for other_pos in range(1, 264):
        if other_pos == position:
            continue
        other_aa = sequence_list[0][other_pos - 1]  # Access the first sequence in sequence_list
        energy_new += J_dict.get((position, other_pos, new_amino_acid, other_aa), 0)

    print(f"E({old_amino_acid}) at {position}: {energy_old}")
    print(f"E({new_amino_acid}) at {position}: {energy_new}")
    print(f"Delta E for {old_amino_acid}{position}{new_amino_acid}: {energy_old - energy_new}")

# # Example usage
# position = 140
# old_amino_acid = 'C'
# new_amino_acid = 'D'
# calculate_delta_e(position, old_amino_acid, new_amino_acid, sequence_list, J_dict)

# 3. find de sign change in top 5 pairs in 1220 sequences:

for the 1220 sequences and top 5 pairs (10mutations in total from above calculation): find all sequences where the DE for either of the single mutations change sign. 

In [ ]:
import csv

# Function to calculate DE for a single mutation
def calculate_de(sequence, position, old_amino_acid, new_amino_acid, J_dict):
    energy_old = 0
    energy_new = 0
    for other_pos in range(1, 264):
        if other_pos == position:
            continue
        other_aa = sequence[other_pos - 1]
        energy_old += J_dict.get((position, other_pos, old_amino_acid, other_aa), 0)
        energy_new += J_dict.get((position, other_pos, new_amino_acid, other_aa), 0)
    return energy_old - energy_new

# Find sequences where DE changes sign
sequences_with_sign_change = []

# Open a TSV file to store the results
with open('sequences_with_sign_change.tsv', 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t')
    writer.writerow(['Sequence Index', 'Full Sequence', 'Mutation', 'DE_Reference', 'DE_Sequence'])

    for seq_index, seq in enumerate(sequence_list):
        for _, row in non_adjacent_strongest_pairs.iterrows():
            first_site = row['First_site']
            second_site = row['Second_site']
            first_mutation = row['First_mutation']
            second_mutation = row['Second_mutation']

            # Extract old and new amino acids for both mutations
            old_aa1, new_aa1 = first_mutation[0], first_mutation[-1]
            old_aa2, new_aa2 = second_mutation[0], second_mutation[-1]

            # Calculate DE for both mutations
            delta_e1 = calculate_de(seq, first_site, old_aa1, new_aa1, J_dict)
            delta_e2 = calculate_de(seq, second_site, old_aa2, new_aa2, J_dict)

            # Check if DE changes sign
            if (delta_e1 * row['First_DE'] < 0) or (delta_e2 * row['Second_DE'] < 0):
                sequences_with_sign_change.append((seq_index, seq))
                # Store the mutation details
                writer.writerow([seq_index, seq, first_mutation, row['First_DE'], delta_e1])
                writer.writerow([seq_index, seq, second_mutation, row['Second_DE'], delta_e2])

# Print all mutations in the specified format
with open('sequences_with_sign_change.tsv', 'r') as tsvfile:
    for line in tsvfile:
        print(line.strip())

# Display the results
print(f"Found {len(set(sequences_with_sign_change))} sequences with DE sign change.")

Sequence Index	Full Sequence	Mutation	DE_Reference	DE_Sequence
0	ABAAABABACBBAACBDBABBDBDBBBAAAAAABAAABDAABAABAAABBBAAAAACACBAABAADACCBADAACACABBABABAAABABBAABDAABCABAABABAAAABDCBCAAADABCCADDAAAAAAAACDBDADBADAAABBABAAAAACBBADAAACCAAABAAABAAABAABABAABABABAAAACCAAAAAAAABBBBCACDCAACDBDCBCAAAAABADDBCACAAAAABAABAAAAAAABBDBDBACCBBBA	C140D	-6.70590850525	5.05515121739154
0	ABAAABABACBBAACBDBABBDBDBBBAAAAAABAAABDAABAABAAABBBAAAAACACBAABAADACCBADAACACABBABABAAABABBAABDAABCABAABABAAAABDCBCAAADABCCADDAAAAAAAACDBDADBADAAABBABAAAAACBBADAAACCAAABAAABAAABAABABAABABABAAAACCAAAAAAAABBBBCACDCAACDBDCBCAAAAABADDBCACAAAAABAABAAAAAAABBDBDBACCBBBA	D148B	-4.5778160966	7.381659193983069
2	ABAAABCBACBBAACBDBABCDBDABBAAACAABAAABDAABAABAAABCBAAAAACACBAABAADACCBAAAACACABBABABAAABABBAABDAABCAAAABABAAAABDCBCAAAAABBCCADAAAAAAAABDBDADBADAAABBABAAAAACBBADAAACCAAABAAABAAABAABABAABABABAAAACCAAAABAABBBBBCACDCAACDBCCCCAAAAABADDBCADAAAAABAABAAAAAAABBABDBACCBBBA	C140D	-6.70590850525	2.1133684122542036
2	ABAAABCBACBBAACBDBABC